In [1]:
import pandas as pd
import pymongo
import json
import datetime as dt

In [2]:
# import all cvs from data folder
earthquake_csv = pd.read_csv("../raw/Earthquake_data.csv")
floods_csv = pd.read_csv("../floods_data.csv")
tornado_csv = pd.read_csv("../tor_output.csv")
hurricane_csv = pd.read_csv("../hurricane_cleaned_df.csv")

/Users/jeremyelam/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
earthquake_df = pd.DataFrame(earthquake_csv)
floods_df = pd.DataFrame(floods_csv)
hurricane_df = pd.DataFrame(hurricane_csv)
tornado_df = pd.DataFrame(tornado_csv)

## EARTHQUAKE DATA CLEANING

In [4]:
earthquake_df["Date"] = earthquake_df.Year.map(str)+'-'+ earthquake_df.Month.map(str) + '-' + earthquake_df.Day.map(str)


In [5]:
earthquake_df.Date = pd.to_datetime(earthquake_df.Date)

In [9]:
earthquake_df = earthquake_df.fillna(0)

In [10]:
earthquake_df = earthquake_df.dropna()

In [11]:
earthquake_df.Hour = earthquake_df.Hour.astype(int)

In [12]:
earthquake_df.Minute = earthquake_df.Minute.astype(int)

In [13]:
earthquake_df.Minute = earthquake_df.Minute.map("{:02}".format)  
earthquake_df.Hour  = earthquake_df.Hour.map("{:02}".format)

In [14]:
earthquake_df["Time"] = earthquake_df.Hour.map(str) + ":" + earthquake_df.Minute

In [15]:
earthquake_df = earthquake_df.rename(columns = {"Latitude": "latitude", "Longitude": "longitude"})

In [16]:
earthquake_df.Time = pd.to_datetime(earthquake_df.Time).dt.time

In [17]:
earthquake_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194 entries, 0 to 193
Data columns (total 19 columns):
Year                           194 non-null int64
Month                          194 non-null int64
Day                            194 non-null int64
Hour                           194 non-null object
Minute                         194 non-null object
Location                       194 non-null object
latitude                       194 non-null float64
longitude                      194 non-null float64
Magnitude                      194 non-null float64
Deaths                         194 non-null float64
Injuries                       194 non-null float64
Damage in Millions             194 non-null float64
Damage Scale                   194 non-null float64
Number of Houses Destroyed     194 non-null float64
Scale of houses destroyed      194 non-null float64
Number of Houses Damaged       194 non-null float64
Scale of houses  damaged       194 non-null float64
Date                

## HURRICANE DATA CLEANING

In [18]:
earthquake_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194 entries, 0 to 193
Data columns (total 19 columns):
Year                           194 non-null int64
Month                          194 non-null int64
Day                            194 non-null int64
Hour                           194 non-null object
Minute                         194 non-null object
Location                       194 non-null object
latitude                       194 non-null float64
longitude                      194 non-null float64
Magnitude                      194 non-null float64
Deaths                         194 non-null float64
Injuries                       194 non-null float64
Damage in Millions             194 non-null float64
Damage Scale                   194 non-null float64
Number of Houses Destroyed     194 non-null float64
Scale of houses destroyed      194 non-null float64
Number of Houses Damaged       194 non-null float64
Scale of houses  damaged       194 non-null float64
Date                

In [19]:
hurricane_df = hurricane_df.drop(columns = ["Unnamed: 0"])

In [20]:
hurricane_df[["Date","Time", "Time_Zone"]] = hurricane_df.iso_time.str.split(" ", expand = True)

In [21]:
hurricane_df.Date = pd.to_datetime(hurricane_df.Date)
hurricane_df.Time = pd.to_datetime(hurricane_df.Time).dt.time

## FLOOD DATA CLEANING

In [22]:
floods_df[["Date", "Time"]] = floods_df.Start_Time.str.split(" ", expand = True)

In [23]:
floods_df.Date = pd.to_datetime(floods_df.Date)
floods_df.Time = pd.to_datetime(floods_df.Time).dt.time

In [24]:
floods_df = floods_df.rename(columns = {"event_latitude": "latitude", "event_longitude": "longitude"})


In [25]:
floods_df = floods_df.drop(columns = ["Start_Date", "End_Time", "End_Date","End_Date","Start_Time"])

In [26]:
floods_df.event_begin_time = pd.to_datetime(floods_df.event_begin_time)
floods_df.event_end_time = pd.to_datetime(floods_df.event_end_time)


## TORNADO DATA CLEANING

In [27]:
tornado_df.Date = pd.to_datetime(tornado_df.Date, errors = 'coerce')
tornado_df.Time = pd.to_datetime(tornado_df.Time, errors = 'coerce')


In [28]:
tornado_df.Property_loss = tornado_df.Property_loss.replace({'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(int)

In [29]:
tornado_df=tornado_df.dropna()

## Change all column to lower case

In [30]:
floods_df.columns = map(str.lower, floods_df.columns)
earthquake_df.columns = map(str.lower, earthquake_df.columns)
hurricane_df.columns = map(str.lower, hurricane_df.columns)
tornado_df.columns = map(str.lower, tornado_df.columns)

## View all DataFrames

In [31]:
floods_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34586 entries, 0 to 34585
Data columns (total 15 columns):
episode_id          34586 non-null int64
state               34586 non-null object
event_type          34586 non-null object
event_begin_time    34586 non-null datetime64[ns]
event_end_time      34586 non-null datetime64[ns]
event_timezone      34586 non-null object
damage_property     34586 non-null float64
latitude            34586 non-null float64
longitude           34586 non-null float64
cz_type             34586 non-null object
cz_fips_code        34586 non-null int64
damage_crops        34586 non-null float64
source              34586 non-null object
date                34586 non-null datetime64[ns]
time                34586 non-null object
dtypes: datetime64[ns](3), float64(4), int64(2), object(6)
memory usage: 4.0+ MB


In [32]:
earthquake_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194 entries, 0 to 193
Data columns (total 19 columns):
year                           194 non-null int64
month                          194 non-null int64
day                            194 non-null int64
hour                           194 non-null object
minute                         194 non-null object
location                       194 non-null object
latitude                       194 non-null float64
longitude                      194 non-null float64
magnitude                      194 non-null float64
deaths                         194 non-null float64
injuries                       194 non-null float64
damage in millions             194 non-null float64
damage scale                   194 non-null float64
number of houses destroyed     194 non-null float64
scale of houses destroyed      194 non-null float64
number of houses damaged       194 non-null float64
scale of houses  damaged       194 non-null float64
date                

In [33]:
hurricane_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57095 entries, 0 to 57094
Data columns (total 14 columns):
index           57095 non-null int64
season          57095 non-null int64
basin           57095 non-null object
subbasin        57095 non-null object
name            57095 non-null object
iso_time        57095 non-null object
nature          57095 non-null object
latitude        57095 non-null float64
longitude       57095 non-null float64
usa_wind        57095 non-null float64
usa_pressure    30937 non-null float64
date            57095 non-null datetime64[ns]
time            57095 non-null object
time_zone       57095 non-null object
dtypes: datetime64[ns](1), float64(4), int64(2), object(7)
memory usage: 6.1+ MB


In [34]:
tornado_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61733 entries, 0 to 66140
Data columns (total 13 columns):
date             61733 non-null datetime64[ns]
time             61733 non-null datetime64[ns]
state            61733 non-null object
fscale           61733 non-null object
injured          61733 non-null float64
fatalities       61733 non-null float64
property_loss    61733 non-null int64
slat             61733 non-null float64
slon             61733 non-null float64
elat             61733 non-null float64
elon             61733 non-null float64
length           61733 non-null float64
width            61733 non-null float64
dtypes: datetime64[ns](2), float64(8), int64(1), object(2)
memory usage: 6.6+ MB


## Convert all the dataframes into json format

In [35]:
hurricane_data = json.loads(hurricane_df.T.to_json()).values()
floods_data = json.loads(floods_df.T.to_json()).values()
earthquake_data = json.loads(earthquake_df.T.to_json()).values()
tornado_data = json.loads(tornado_df.T.to_json()).values()

## NoSQL

In [36]:
# connect to mongod local server
conn = 'mongodb://localhost:27017'
# create client 
client = pymongo.MongoClient(conn)
# create database
# db.weather_data.drop()
db = client.weather_data


In [37]:
db.hurricane_data.drop()
db.hurricane_data.insert(hurricane_data)

/Users/jeremyelam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[ObjectId('5e1a590a375d0a76eaed57dc'),
 ObjectId('5e1a590a375d0a76eaed57dd'),
 ObjectId('5e1a590a375d0a76eaed57de'),
 ObjectId('5e1a590a375d0a76eaed57df'),
 ObjectId('5e1a590a375d0a76eaed57e0'),
 ObjectId('5e1a590a375d0a76eaed57e1'),
 ObjectId('5e1a590a375d0a76eaed57e2'),
 ObjectId('5e1a590a375d0a76eaed57e3'),
 ObjectId('5e1a590a375d0a76eaed57e4'),
 ObjectId('5e1a590a375d0a76eaed57e5'),
 ObjectId('5e1a590a375d0a76eaed57e6'),
 ObjectId('5e1a590a375d0a76eaed57e7'),
 ObjectId('5e1a590a375d0a76eaed57e8'),
 ObjectId('5e1a590a375d0a76eaed57e9'),
 ObjectId('5e1a590a375d0a76eaed57ea'),
 ObjectId('5e1a590a375d0a76eaed57eb'),
 ObjectId('5e1a590a375d0a76eaed57ec'),
 ObjectId('5e1a590a375d0a76eaed57ed'),
 ObjectId('5e1a590a375d0a76eaed57ee'),
 ObjectId('5e1a590a375d0a76eaed57ef'),
 ObjectId('5e1a590a375d0a76eaed57f0'),
 ObjectId('5e1a590a375d0a76eaed57f1'),
 ObjectId('5e1a590a375d0a76eaed57f2'),
 ObjectId('5e1a590a375d0a76eaed57f3'),
 ObjectId('5e1a590a375d0a76eaed57f4'),
 ObjectId('5e1a590a375d0a

In [38]:
db.flood_data.drop()
db.flood_data.insert(floods_data)

/Users/jeremyelam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[ObjectId('5e1a590b375d0a76eaee36e3'),
 ObjectId('5e1a590b375d0a76eaee36e4'),
 ObjectId('5e1a590b375d0a76eaee36e5'),
 ObjectId('5e1a590b375d0a76eaee36e6'),
 ObjectId('5e1a590b375d0a76eaee36e7'),
 ObjectId('5e1a590b375d0a76eaee36e8'),
 ObjectId('5e1a590b375d0a76eaee36e9'),
 ObjectId('5e1a590b375d0a76eaee36ea'),
 ObjectId('5e1a590b375d0a76eaee36eb'),
 ObjectId('5e1a590b375d0a76eaee36ec'),
 ObjectId('5e1a590b375d0a76eaee36ed'),
 ObjectId('5e1a590b375d0a76eaee36ee'),
 ObjectId('5e1a590b375d0a76eaee36ef'),
 ObjectId('5e1a590b375d0a76eaee36f0'),
 ObjectId('5e1a590b375d0a76eaee36f1'),
 ObjectId('5e1a590b375d0a76eaee36f2'),
 ObjectId('5e1a590b375d0a76eaee36f3'),
 ObjectId('5e1a590b375d0a76eaee36f4'),
 ObjectId('5e1a590b375d0a76eaee36f5'),
 ObjectId('5e1a590b375d0a76eaee36f6'),
 ObjectId('5e1a590b375d0a76eaee36f7'),
 ObjectId('5e1a590b375d0a76eaee36f8'),
 ObjectId('5e1a590b375d0a76eaee36f9'),
 ObjectId('5e1a590b375d0a76eaee36fa'),
 ObjectId('5e1a590b375d0a76eaee36fb'),
 ObjectId('5e1a590b375d0a

In [39]:
db.earthquake_data.drop()
db.earthquake_data.insert(earthquake_data)

/Users/jeremyelam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[ObjectId('5e1a590b375d0a76eaeebdfd'),
 ObjectId('5e1a590b375d0a76eaeebdfe'),
 ObjectId('5e1a590b375d0a76eaeebdff'),
 ObjectId('5e1a590b375d0a76eaeebe00'),
 ObjectId('5e1a590b375d0a76eaeebe01'),
 ObjectId('5e1a590b375d0a76eaeebe02'),
 ObjectId('5e1a590b375d0a76eaeebe03'),
 ObjectId('5e1a590b375d0a76eaeebe04'),
 ObjectId('5e1a590b375d0a76eaeebe05'),
 ObjectId('5e1a590b375d0a76eaeebe06'),
 ObjectId('5e1a590b375d0a76eaeebe07'),
 ObjectId('5e1a590b375d0a76eaeebe08'),
 ObjectId('5e1a590b375d0a76eaeebe09'),
 ObjectId('5e1a590b375d0a76eaeebe0a'),
 ObjectId('5e1a590b375d0a76eaeebe0b'),
 ObjectId('5e1a590b375d0a76eaeebe0c'),
 ObjectId('5e1a590b375d0a76eaeebe0d'),
 ObjectId('5e1a590b375d0a76eaeebe0e'),
 ObjectId('5e1a590b375d0a76eaeebe0f'),
 ObjectId('5e1a590b375d0a76eaeebe10'),
 ObjectId('5e1a590b375d0a76eaeebe11'),
 ObjectId('5e1a590b375d0a76eaeebe12'),
 ObjectId('5e1a590b375d0a76eaeebe13'),
 ObjectId('5e1a590b375d0a76eaeebe14'),
 ObjectId('5e1a590b375d0a76eaeebe15'),
 ObjectId('5e1a590b375d0a

In [40]:
db.tornado_data.drop()
db.tornado_data.insert(tornado_data)

/Users/jeremyelam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[ObjectId('5e1a590b375d0a76eaeebebf'),
 ObjectId('5e1a590b375d0a76eaeebec0'),
 ObjectId('5e1a590b375d0a76eaeebec1'),
 ObjectId('5e1a590b375d0a76eaeebec2'),
 ObjectId('5e1a590b375d0a76eaeebec3'),
 ObjectId('5e1a590b375d0a76eaeebec4'),
 ObjectId('5e1a590b375d0a76eaeebec5'),
 ObjectId('5e1a590b375d0a76eaeebec6'),
 ObjectId('5e1a590b375d0a76eaeebec7'),
 ObjectId('5e1a590b375d0a76eaeebec8'),
 ObjectId('5e1a590b375d0a76eaeebec9'),
 ObjectId('5e1a590b375d0a76eaeebeca'),
 ObjectId('5e1a590b375d0a76eaeebecb'),
 ObjectId('5e1a590b375d0a76eaeebecc'),
 ObjectId('5e1a590b375d0a76eaeebecd'),
 ObjectId('5e1a590b375d0a76eaeebece'),
 ObjectId('5e1a590b375d0a76eaeebecf'),
 ObjectId('5e1a590b375d0a76eaeebed0'),
 ObjectId('5e1a590b375d0a76eaeebed1'),
 ObjectId('5e1a590b375d0a76eaeebed2'),
 ObjectId('5e1a590b375d0a76eaeebed3'),
 ObjectId('5e1a590b375d0a76eaeebed4'),
 ObjectId('5e1a590b375d0a76eaeebed5'),
 ObjectId('5e1a590b375d0a76eaeebed6'),
 ObjectId('5e1a590b375d0a76eaeebed7'),
 ObjectId('5e1a590b375d0a